# Hands on Object Relational Mapping in PyCram

This tutorial will walk through the serialization of a minimal plan in pycram.
First we will import sqlalchemy, create an in memory database and connect a session to it.

In [1]:
import sqlalchemy
import sqlalchemy.orm

engine = sqlalchemy.create_engine("sqlite+pysqlite:///:memory:", echo=False)
session = sqlalchemy.orm.Session(bind=engine)
session

Next we create the database schema using the sqlalchemy functionality. For that we need to import the base class of pycram.orm.

In [2]:
import pycram.orm.base
pycram.orm.base.Base.metadata.create_all(engine)
session.commit()

pybullet build time: Sep 20 2021 20:33:29
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
/home/tom/catkin_ws/src/pycram-1/src/pycram/orm/action_designator.py:23: SAWarning: Implicitly combining column Action.id with column ParkArms.id under attribute 'id'.  Please configure one or more attributes for these same-named columns explicitly.
  class ParkArmsAction(Action):
/home/to

Next we will write a simple plan where the robot parks his arms and then moves somewhere. We will construct a TaskTree around it such that we can serialize it later. As usual, we first create a world and then define the plan. After that we get and print the task tree.

In [3]:
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.process_module import simulated_robot
from pycram.resolver.plans import Arms
from pycram.task import with_tree
import pycram.task
from pycram.bullet_world import BulletWorld, Object
from pycram.robot_descriptions.robot_description_handler import InitializedRobotDescription as robot_description
import anytree

world = BulletWorld()
robot = Object(robot_description.i.name, "robot", robot_description.i.name + ".urdf")
kitchen = Object("kitchen", "environment", "kitchen.urdf")
milk = Object("milk", "milk", "milk.stl", position=[1.3, 1, 0.9])
cereal = Object("cereal", "cereal", "breakfast_cereal.stl", position=[1.3, 0.7, 0.95])

@with_tree
def plan():
    with simulated_robot:
        ActionDesignator(ParkArmsAction(Arms.BOTH)).perform()

        location = LocationDesignator(CostmapLocation(target=milk, reachable_for=robot))
        pose = location.reference()
        ActionDesignator(
            NavigateAction(target_position=pose["position"], target_orientation=pose["orientation"])).perform()

plan()
world.exit()

task_tree = pycram.task.task_tree
print(anytree.RenderTree(task_tree))



startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 4070 Ti/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 525.105.17
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 525.105.17
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 4070 Ti/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link

Next we serialize the task tree by just recursively inserting from its root.

In [4]:
task_tree.root.insert(session)

pycram.orm.task.TaskTreeNode(1, 1, 2023-04-10 14:46:53.763427, None, RUNNING, None)

Lastly we can look at various table to see how the structures got logged.

In [5]:
import pycram.orm.task
import pycram.orm.action_designator

navigations = session.query(pycram.orm.action_designator.NavigateAction).all()
print(*navigations)

nodes = session.query(pycram.orm.task.TaskTreeNode).all()
print(*nodes)


pycram.orm.action_designator.NavigateAction(2, Navigate, 1, 1, 2)
pycram.orm.task.TaskTreeNode(1, 1, 2023-04-10 14:46:53.763427, None, RUNNING, None) pycram.orm.task.TaskTreeNode(2, 2, 2023-04-10 14:47:03.344791, 2023-04-10 14:47:04.507549, SUCCEEDED, 1) pycram.orm.task.TaskTreeNode(3, 3, 2023-04-10 14:47:03.344834, 2023-04-10 14:47:03.845934, SUCCEEDED, 2) pycram.orm.task.TaskTreeNode(4, 4, 2023-04-10 14:47:04.005707, 2023-04-10 14:47:04.507543, SUCCEEDED, 2)
